# 라이브러리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook
#import seaborn as sns

from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree

from sklearn.model_selection import train_test_split

import graphviz

import time

# Load data
 - HN16_ALL.sas7bdat : 설문지 데이터

In [2]:
df_data = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
print(df_data.shape)
df_data.head()

(8150, 768)


,mod_d,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,...,N_VA_RAE,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,LF_secur_y,LF_BUYER,LF_SAFE
0,2018.02.01.,A651172801,A6511728,2016.0,1.0,1.0,2.0,A651,1.0,61.0,...,68.125541,817.506495,0.000000,1.812120,1.599025,17.241284,417.773500,1.0,1.0,1.0
1,2018.02.01.,A651172802,A6511728,2016.0,1.0,1.0,2.0,A651,2.0,57.0,...,91.302698,1095.561039,0.005945,1.075103,0.756771,5.302599,31.845311,1.0,1.0,1.0
2,2018.02.01.,A651183001,A6511830,2016.0,1.0,1.0,2.0,A651,2.0,41.0,...,261.852637,2147.567940,82.888642,1.093273,0.939505,16.655787,29.051934,1.0,2.0,2.0
3,2018.02.01.,A651183002,A6511830,2016.0,1.0,1.0,2.0,A651,1.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018.02.01.,A651183003,A6511830,2016.0,1.0,1.0,2.0,A651,2.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 데이터 전처리

In [3]:
df_data_cut = pd.DataFrame(df_data, columns=[
    df_data.HE_HP.name ,
    df_data.region.name ,
    df_data.sex.name ,
    df_data.age.name ,
    df_data.incm.name ,
    df_data.ho_incm.name ,
    df_data.edu.name ,
    df_data.occp.name ,
    df_data.cfam.name ,
    df_data.allownc.name ,
    df_data.house.name ,
    df_data.live_t.name ,
    df_data.ainc_1.name ,
    df_data.ainc.name ,
    df_data.marri_1.name ,
    df_data.marri_2.name ,
    df_data.tins.name ,
    df_data.npins.name ,
    df_data.educ.name ,
    df_data.graduat.name ,
    df_data.EC1_1.name ,
    df_data.EC1_2.name ,
    df_data.EC_occp.name ,
    df_data.EC_stt_1.name ,
    df_data.EC_stt_2.name ,
    df_data.EC_wh.name ,
    df_data.EC_wht_0.name ,
    df_data.EC_wht_23.name ,
    df_data.EC_wht_5.name ,
    df_data.EC_lgw_2.name ,
    df_data.EC_lgw_4.name ,
    df_data.EC_lgw_5.name ,
    df_data.EC_pedu_1.name ,
    df_data.EC_pedu_2.name
])
total_column_count = df_data_cut.columns.size
print(total_column_count)
print(df_data_cut.shape)
df_data_cut.head()

34
(8150, 34)


,HE_HP,region,sex,age,incm,ho_incm,edu,occp,cfam,allownc,...,EC_stt_2,EC_wh,EC_wht_0,EC_wht_23,EC_wht_5,EC_lgw_2,EC_lgw_4,EC_lgw_5,EC_pedu_1,EC_pedu_2
0,3.0,1.0,1.0,61.0,3.0,3.0,4.0,7.0,3.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,3.0,3.0
1,2.0,1.0,2.0,57.0,3.0,3.0,4.0,6.0,3.0,20.0,...,2.0,1.0,2.0,50.0,1.0,3.0,1.0,1.0,3.0,3.0
2,1.0,1.0,2.0,41.0,3.0,3.0,4.0,1.0,5.0,20.0,...,2.0,2.0,2.0,24.0,1.0,2.0,1.0,1.0,8.0,5.0
3,3.0,1.0,1.0,39.0,3.0,3.0,4.0,3.0,5.0,20.0,...,1.0,1.0,1.0,61.0,1.0,5.0,1.0,1.0,8.0,5.0
4,1.0,1.0,2.0,38.0,3.0,3.0,4.0,7.0,5.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,5.0,5.0


## 고혈압 유병여부 NaN값 대상 삭제

In [4]:
df_data_cut.dropna(subset=['HE_HP'], inplace=True)
df_data_cut.shape

(6105, 34)

## 모든 값이 NaN인 경우 삭제

In [5]:
df_data_cut.dropna(how='all', inplace=True)
df_data_cut.reindex()
print(df_data_cut.shape)
df_data_cut.head()

(6105, 34)


,HE_HP,region,sex,age,incm,ho_incm,edu,occp,cfam,allownc,...,EC_stt_2,EC_wh,EC_wht_0,EC_wht_23,EC_wht_5,EC_lgw_2,EC_lgw_4,EC_lgw_5,EC_pedu_1,EC_pedu_2
0,3.0,1.0,1.0,61.0,3.0,3.0,4.0,7.0,3.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,3.0,3.0
1,2.0,1.0,2.0,57.0,3.0,3.0,4.0,6.0,3.0,20.0,...,2.0,1.0,2.0,50.0,1.0,3.0,1.0,1.0,3.0,3.0
2,1.0,1.0,2.0,41.0,3.0,3.0,4.0,1.0,5.0,20.0,...,2.0,2.0,2.0,24.0,1.0,2.0,1.0,1.0,8.0,5.0
3,3.0,1.0,1.0,39.0,3.0,3.0,4.0,3.0,5.0,20.0,...,1.0,1.0,1.0,61.0,1.0,5.0,1.0,1.0,8.0,5.0
4,1.0,1.0,2.0,38.0,3.0,3.0,4.0,7.0,5.0,20.0,...,8.0,8.0,8.0,888.0,88.0,3.0,1.0,1.0,5.0,5.0


## 특정 value 값을 NaN 처리

In [6]:
def setNanValues(in_df, name, nan_values) :
    
    start_na_count = in_df[name].isna().sum()
    
    uniq_values = sorted(map(float,in_df[name].dropna().unique()))
    
    in_df[name] = in_df[name].map(lambda x: np.nan if x in nan_values else x)
        
    #print(sorted(map(float,in_df[name].dropna().unique())))
        
    end_na_count = in_df[name].isna().sum()
    
    print("%s NaN count %s -> %s" % (name,start_na_count,end_na_count))
    
    return in_df

nan_df_data = df_data_cut.copy(deep=True)
nan_df_data = setNanValues(nan_df_data, nan_df_data.marri_1.name, [9.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.marri_2.name, [88.0, 99.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.EC_pedu_1.name, [88.0, 99.0])
nan_df_data = setNanValues(nan_df_data, nan_df_data.EC_pedu_2.name, [88.0, 99.0])
print(nan_df_data.shape)
nan_df_data.columns

marri_1 NaN count 0 -> 0
marri_2 NaN count 0 -> 941
EC_pedu_1 NaN count 0 -> 1042
EC_pedu_2 NaN count 0 -> 890
(6105, 34)


Index(['HE_HP', 'region', 'sex', 'age', 'incm', 'ho_incm', 'edu', 'occp',
       'cfam', 'allownc', 'house', 'live_t', 'ainc_1', 'ainc', 'marri_1',
       'marri_2', 'tins', 'npins', 'educ', 'graduat', 'EC1_1', 'EC1_2',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'EC_wh', 'EC_wht_0', 'EC_wht_23',
       'EC_wht_5', 'EC_lgw_2', 'EC_lgw_4', 'EC_lgw_5', 'EC_pedu_1',
       'EC_pedu_2'],
      dtype='object')

## NaN 값 처리

In [7]:
'''
strategy = mean, median, most_frequent, constant
'''
def setNanImputer(in_df, name, strategy="mean") :
    
    tmp_df = pd.DataFrame(in_df[name], columns=[name])
    
    imp = SimpleImputer(missing_values=np.nan, strategy=strategy)
    in_df[name] = imp.fit_transform(tmp_df)
        
    print('%s null count %s, na count %s' % (name, in_df[name].isnull().sum(), in_df[name].isna().sum()))
    
    return in_df


imp_df = nan_df_data.copy(deep=True)

imputer_infos = [
      ['ainc', 'mean']
]

for name, strategy in imputer_infos :
    imp_df = setNanImputer(imp_df, name, strategy=strategy)

ainc null count 0, na count 0


## One-Hot Encoding

In [8]:
def addOneHotEncodingColumns(in_df, name) :
    
    uniq_values = sorted(map(float,in_df[name].dropna().unique()))
    
    dummie = pd.get_dummies(in_df[name], dummy_na=True)
    dummie.columns=['%s_ONE_%s' % (name,str(n)) for n in uniq_values] + ['%s_ONE_NaN' % name]
    
    one_hot_df = pd.concat([in_df,dummie],axis=1)    
    one_hot_df.drop(columns=[name], inplace=True)
    
    return one_hot_df

one_hot_df = imp_df.copy(deep=True)
print(one_hot_df.shape)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.region.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.incm.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.ho_incm.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.edu.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.occp.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.EC_pedu_1.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.EC_pedu_2.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.marri_1.name)
one_hot_df = addOneHotEncodingColumns(one_hot_df, nan_df_data.marri_2.name)
print(one_hot_df.shape)
print(one_hot_df.columns)
one_hot_df.tail()

(6105, 34)
(6105, 93)
Index(['HE_HP', 'sex', 'age', 'cfam', 'allownc', 'house', 'live_t', 'ainc_1',
       'ainc', 'tins', 'npins', 'educ', 'graduat', 'EC1_1', 'EC1_2', 'EC_occp',
       'EC_stt_1', 'EC_stt_2', 'EC_wh', 'EC_wht_0', 'EC_wht_23', 'EC_wht_5',
       'EC_lgw_2', 'EC_lgw_4', 'EC_lgw_5', 'region_ONE_1.0', 'region_ONE_2.0',
       'region_ONE_3.0', 'region_ONE_4.0', 'region_ONE_5.0', 'region_ONE_6.0',
       'region_ONE_7.0', 'region_ONE_8.0', 'region_ONE_9.0', 'region_ONE_10.0',
       'region_ONE_11.0', 'region_ONE_12.0', 'region_ONE_13.0',
       'region_ONE_14.0', 'region_ONE_15.0', 'region_ONE_16.0',
       'region_ONE_17.0', 'region_ONE_NaN', 'incm_ONE_1.0', 'incm_ONE_2.0',
       'incm_ONE_3.0', 'incm_ONE_4.0', 'incm_ONE_NaN', 'ho_incm_ONE_1.0',
       'ho_incm_ONE_2.0', 'ho_incm_ONE_3.0', 'ho_incm_ONE_4.0',
       'ho_incm_ONE_NaN', 'edu_ONE_1.0', 'edu_ONE_2.0', 'edu_ONE_3.0',
       'edu_ONE_4.0', 'edu_ONE_NaN', 'occp_ONE_1.0', 'occp_ONE_2.0',
       'occp_ONE_3.0', 

,HE_HP,sex,age,cfam,allownc,house,live_t,ainc_1,ainc,tins,...,EC_pedu_2_ONE_NaN,marri_1_ONE_1.0,marri_1_ONE_2.0,marri_1_ONE_NaN,marri_2_ONE_1.0,marri_2_ONE_2.0,marri_2_ONE_3.0,marri_2_ONE_4.0,marri_2_ONE_9.0,marri_2_ONE_NaN
8145,3.0,1.0,75.0,3.0,10.0,2.0,2.0,150.0,150.0,20.0,...,1,1,0,0,1,0,0,0,0,0
8146,3.0,2.0,72.0,3.0,10.0,2.0,2.0,150.0,150.0,20.0,...,0,1,0,0,1,0,0,0,0,0
8147,1.0,2.0,54.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,1,0,0,1,0,0,0,0,0
8148,3.0,1.0,56.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,1,0,0,1,0,0,0,0,0
8149,1.0,2.0,28.0,4.0,20.0,3.0,2.0,12000.0,1000.0,20.0,...,0,0,1,0,0,0,0,0,0,1


## 결측치가 하나 이상 있는 컬럼 확인

In [9]:
nan_names = one_hot_df.columns[one_hot_df.isna().any()]
nan_names

Index([], dtype='object')

In [10]:
for index, name in enumerate(nan_names) :
    value_count = pd.value_counts(one_hot_df[name], dropna=False)
    nan_count = value_count[np.nan]
    print("[ %3d ] %s NaN count %s" % (index,name,nan_count))
    value_count.sort_index().plot(kind='bar', figsize=(15,5))
    plt.show()

# Train, Test data split

In [11]:
train_df, test_df = train_test_split(one_hot_df, test_size=0.33)
print(train_df.shape)
print(test_df.shape)

(4090, 93)
(2015, 93)


# Model

In [12]:
def runDTR(x_name_list, y_name, train_df, test_df, max_depth=None) :
    
    stime = time.time()

    x_train_df = train_df[x_name_list]
    y_train_df = train_df[y_name]
    
    x_test_df = test_df[x_name_list]
    y_test_df = test_df[y_name]

    model = DecisionTreeClassifier(
          criterion = 'gini'
        , max_depth=max_depth
    ).fit(x_train_df,y_train_df)
    
    pdf_name = "./pdf_decision_tree_classifier_gini/target-%s_feature_count-%s_max_depth-%s" % (y_name,x_name_list.__len__(),max_depth)

    dot_data = tree.export_graphviz(model,
                                    out_file=None, 
                                    feature_names=x_name_list, 
                                    # class_names=['정상', '고혈압전단계', '고혈압']
                                    class_names=['1','2','3']
                                   )

    graph = graphviz.Source(dot_data) 
    graph.render(pdf_name)
    #graph.render("%s+%s" % (x_name,y_name))

    modelPrediction = model.predict(x_test_df)
    accuracyRate = accuracy_score(
        y_true=y_test_df
    ,   y_pred=modelPrediction
    )

    etime = time.time()
    
    print(pdf_name,', 정확도(%s) 소요시간(%s)' % (accuracyRate,etime-stime))
    
    return accuracyRate

In [13]:
Y = one_hot_df.HE_HP.name
X = one_hot_df.columns.drop(Y).tolist()


for depth in range(one_hot_df.columns.size, 5, -1) :
    accuracy_rate = runDTR(X, Y, train_df, test_df, depth)

./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-93 , 정확도(0.4674937965260546) 소요시간(3.7915420532226562)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-92 , 정확도(0.46997518610421835) 소요시간(3.4352543354034424)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-91 , 정확도(0.46153846153846156) 소요시간(3.3955626487731934)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-90 , 정확도(0.47444168734491315) 소요시간(3.6499762535095215)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-89 , 정확도(0.4660049627791563) 소요시간(3.4548118114471436)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-88 , 정확도(0.4655086848635236) 소요시간(3.471761465072632)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-87 , 정확도(0.4645161290322581) 소요시간(3.5270352363586426)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-86 , 정확도(0.4

./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-29 , 정확도(0.4665012406947891) 소요시간(3.5252580642700195)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-28 , 정확도(0.4640198511166253) 소요시간(3.5340657234191895)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-27 , 정확도(0.4630272952853598) 소요시간(3.585238218307495)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-26 , 정확도(0.4620347394540943) 소요시간(3.557469606399536)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-25 , 정확도(0.4635235732009926) 소요시간(3.551736831665039)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-24 , 정확도(0.47146401985111663) 소요시간(3.419614791870117)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-23 , 정확도(0.4630272952853598) 소요시간(3.485285520553589)
./pdf_decision_tree_classifier_gini/target-HE_HP_feature_count-92_max_depth-22 , 정확도(0.4620347